**Raw Zone**

It has 3 Tables.
- Account: Database dataverse_salestrial_xxx. It's pushed from Dynamics Sales via Synapse link for Dataverse.
- xxx_their_sales: Database dataverse_dataopsdefau_xxx. It's pushed from Dataverse via Synapse link for Dataverse. It was loaded to dataverse from a csv file.
- our_sales: Database raw. It's loaded from a csv file using notebook script.

**Prepare RAW**

In [6]:
%%sql
CREATE DATABASE raw;

StatementMeta(, 5, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [21]:
%%sql
drop table standardised.our_sales

StatementMeta(sparkpool, 9, 20, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [20]:
%%pyspark

spark.conf.set("fs.azure.account.auth.type", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type", "com.microsoft.azure.synapse.tokenlibrary.ConfBasedSASProvider")
spark.conf.set("spark.storage.synapse.sas", "SAS Token")

our_sales_file_path = "abfss://synfs@dataopsstg01.dfs.core.windows.net/our_sales.csv"
our_sales_raw_table_path = "abfss://raw@dataopsstg01.dfs.core.windows.net/our_sales"

df = spark.read.csv(our_sales_file_path, header=True)
df.show()

df.write.option('path', our_sales_raw_table_path).saveAsTable("raw.our_sales")

StatementMeta(sparkpool, 9, 19, Finished, Available)

+----------+---------------+------------+-------------+----------------+--------+--------------+----------+------+--------+-------------------+-----+-----------+----------+--------------+------------+------------+----------------+-------------+---------------+-------------------+---------------+----------------+-----------+----------------+--------+---------+
| OrderDate|       Category|        City|      Country|    CustomerName|Discount|       OrderID|PostalCode|Profit|Quantity|            Account|Sales|    Segment|  ShipDate|      ShipMode|       State|Sub_Category|DaystoShipActual|SalesForecast|     ShipStatus|DaystoShipScheduled|OrderProfitable|SalesperCustomer|ProfitRatio|SalesaboveTarget|latitude|longitude|
+----------+---------------+------------+-------------+----------------+--------+--------------+----------+------+--------+-------------------+-----+-----------+----------+--------------+------------+------------+----------------+-------------+---------------+----------------

**Prepare Standardised**

In [22]:
%%pyspark
from pyspark.sql.functions import to_date

spark.conf.set("fs.azure.account.auth.type", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type", "com.microsoft.azure.synapse.tokenlibrary.ConfBasedSASProvider")
spark.conf.set("spark.storage.synapse.sas", "SAS Token")

df = spark.sql("SELECT * FROM `raw`.`our_sales`")

df = df.select('OrderDate','City','State','Country','Account','Sales')
date_format = "d/MM/yyyy"
df = df.withColumn("OrderDate", to_date("OrderDate", date_format))

df.show()

our_sales_standardised_table_path = "abfss://standardised@dataopsstg01.dfs.core.windows.net/our_sales"

df.write.option('path', our_sales_standardised_table_path).saveAsTable("standardised.our_sales")

StatementMeta(sparkpool, 9, 21, Finished, Available)

+----------+------------+------------+-------------+-------------------+-----+
| OrderDate|        City|       State|      Country|            Account|Sales|
+----------+------------+------------+-------------+-------------------+-----+
|2011-01-04|     Houston|       Texas|United States|  Northwind Traders|   16|
|2011-01-05|  Naperville|    Illinois|United States|  Northwind Traders|   12|
|2011-01-05|  Naperville|    Illinois|United States|  Northwind Traders|    4|
|2011-01-05|  Naperville|    Illinois|United States|  Northwind Traders|  273|
|2011-01-06|Philadelphia|Pennsylvania|United States|A Datum Corporation|   20|
|2011-01-07|      Athens|     Georgia|United States|       Fabrikam Inc|   13|
|2011-01-07|   Henderson|    Kentucky|United States|       Fabrikam Inc|   31|
|2011-01-07|   Henderson|    Kentucky|United States|       Fabrikam Inc|  610|
|2011-01-07|   Henderson|    Kentucky|United States|       Fabrikam Inc|    5|
|2011-01-07|   Henderson|    Kentucky|United States|

In [26]:
%%pyspark
from pyspark.sql.functions import regexp_replace

df = spark.sql("SELECT name, address1_city,address1_stateorprovince, address1_country FROM `dataverse_salestrial_unqd3841b9b46bf426aa84737d42b716`.`account`")
df = df.withColumn('name', regexp_replace('name', ',', ''))
df = df.withColumn('name', regexp_replace('name', '\\.', ''))

spark.conf.set("fs.azure.account.auth.type", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type", "com.microsoft.azure.synapse.tokenlibrary.ConfBasedSASProvider")
spark.conf.set("spark.storage.synapse.sas", "SAS Token")

account_standardised_table_path = "abfss://standardised@dataopsstg01.dfs.core.windows.net/account"

df.write.option('path', account_standardised_table_path).saveAsTable("standardised.account")

StatementMeta(sparkpool, 5, 30, Finished, Available)

+-------------------+-------------+------------------------+----------------+
|               name|address1_city|address1_stateorprovince|address1_country|
+-------------------+-------------+------------------------+----------------+
|       Fabrikam Inc|     Lynnwood|               Tennessee|   United States|
|  Northwind Traders|     Columbus|                    Ohio|   United States|
|A Datum Corporation|      Redmond|              Washington|   United States|
|   Alpine Ski House|       Dallas|                   Texas|   United States|
|      Trey Research|San Francisco|              California|   United States|
+-------------------+-------------+------------------------+----------------+



In [37]:
%%pyspark
from pyspark.sql.functions import to_date

spark.conf.set("fs.azure.account.auth.type", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type", "com.microsoft.azure.synapse.tokenlibrary.ConfBasedSASProvider")
spark.conf.set("spark.storage.synapse.sas", "SAS Token")

df = spark.sql("SELECT * FROM `dataverse_dataopsdefau_unq042b19474c3144c393404d6c48e70`.`cr673_their_sales`")

df = df.select('cr673_OrderDate','cr673_City','cr673_State','cr673_Country','cr673_Account','cr673_Sales')
date_format = "yyyy-MM-d"
df = df.withColumn("cr673_OrderDate", to_date("cr673_OrderDate", date_format))

df.show()

our_sales_standardised_table_path = "abfss://standardised@dataopsstg01.dfs.core.windows.net/their_sales"

df.write.option('path', our_sales_standardised_table_path).saveAsTable("standardised.their_sales")

StatementMeta(sparkpool, 9, 36, Finished, Available)

+---------------+--------------+--------------+-------------+-------------------+-----------+
|cr673_OrderDate|    cr673_City|   cr673_State|cr673_Country|      cr673_Account|cr673_Sales|
+---------------+--------------+--------------+-------------+-------------------+-----------+
|     2011-01-07|     Henderson|      Kentucky|United States|       Fabrikam Inc|       2574|
|     2011-01-07|     Henderson|      Kentucky|United States|       Fabrikam Inc|        392|
|     2011-01-08|    Huntsville|         Texas|United States|  Northwind Traders|         77|
|     2011-01-07|     Henderson|      Kentucky|United States|       Fabrikam Inc|        756|
|     2011-01-12|         Dover|      Delaware|United States|A Datum Corporation|         10|
|     2011-01-14|Mount Pleasant|South Carolina|United States|       Fabrikam Inc|        546|
|     2011-01-11|   Springfield|      Virginia|United States|       Fabrikam Inc|         52|
|     2011-01-10|        Laredo|         Texas|United States

**Curated Layer**

In [14]:
%%sql
CREATE DATABASE curated;

StatementMeta(sparkpool, 9, 13, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [17]:
%%pyspark
from pyspark.sql.functions import regexp_replace

spark.conf.set("fs.azure.account.auth.type", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type", "com.microsoft.azure.synapse.tokenlibrary.ConfBasedSASProvider")
spark.conf.set("spark.storage.synapse.sas", "SAS Token")

df = spark.sql("SELECT name, address1_city,address1_stateorprovince, address1_country FROM `standardised`.`account`")
df = df.withColumnRenamed("address1_city", "city")\
       .withColumnRenamed("address1_stateorprovince", "stateorprovince")\
       .withColumnRenamed("address1_country", "country")\

df.show()
account_curated_table_path = "abfss://curated@dataopsstg01.dfs.core.windows.net/account"

df.write.option('path', account_curated_table_path).saveAsTable("curated.account")

StatementMeta(sparkpool, 9, 16, Finished, Available)

+-------------------+-------------+---------------+-------------+
|               name|         city|stateorprovince|      country|
+-------------------+-------------+---------------+-------------+
|       Fabrikam Inc|     Lynnwood|      Tennessee|United States|
|  Northwind Traders|     Columbus|           Ohio|United States|
|A Datum Corporation|      Redmond|     Washington|United States|
|   Alpine Ski House|       Dallas|          Texas|United States|
|      Trey Research|San Francisco|     California|United States|
+-------------------+-------------+---------------+-------------+



In [60]:
%%pyspark
from pyspark.sql.functions import to_date
from pyspark.sql.functions import date_trunc

df_our = spark.sql("SELECT * FROM `standardised`.`our_sales`")
df_our = df_our.withColumn('OrderDate', date_trunc('month', df_our['OrderDate']))

df_their = spark.sql("SELECT * FROM `standardised`.`their_sales`")
df_their = df_their.withColumnRenamed("cr673_OrderDate", "OrderDate")
df_their = df_their.withColumn('OrderDate', date_trunc('month', df_their['OrderDate']))


StatementMeta(sparkpool, 9, 59, Finished, Available)

In [64]:
df_our = df_our.orderBy('OrderDate')
df_their = df_their.orderBy('OrderDate')

merged_df = df_our.join(df_their, on='OrderDate', how='inner')

merged_df = merged_df.withColumnRenamed("City", "Our_City")\
       .withColumnRenamed("State", "Our_State")\
       .withColumnRenamed("Country", "Our_Country")\
       .withColumnRenamed("Sales", "Our_Sales")\
       .withColumnRenamed("cr673_City", "Their_City")\
       .withColumnRenamed("cr673_State", "Their_State")\
       .withColumnRenamed("cr673_Country", "Their_Country")\
       .withColumnRenamed("cr673_Account", "Their_Account")\
       .withColumnRenamed("cr673_Sales", "Their_Sales")\

merged_df.show()

sales_curated_table_path = "abfss://curated@dataopsstg01.dfs.core.windows.net/sales"

merged_df.write.option('path', sales_curated_table_path).saveAsTable("curated.sales")

StatementMeta(sparkpool, 9, 63, Finished, Available)

+-------------------+--------+---------+-------------+-----------------+---------+-------------+------------+-------------+-------------------+-----------+
|          OrderDate|Our_City|Our_State|  Our_Country|          Account|Our_Sales|   Their_City| Their_State|Their_Country|      Their_Account|Their_Sales|
+-------------------+--------+---------+-------------+-----------------+---------+-------------+------------+-------------+-------------------+-----------+
|2011-01-01 00:00:00| Houston|    Texas|United States|Northwind Traders|       16|   Alexandria|    Virginia|United States|       Fabrikam Inc|        155|
|2011-01-01 00:00:00| Houston|    Texas|United States|Northwind Traders|       16|   Alexandria|    Virginia|United States|       Fabrikam Inc|        188|
|2011-01-01 00:00:00| Houston|    Texas|United States|Northwind Traders|       16|   Rapid City|South Dakota|United States|  Northwind Traders|        142|
|2011-01-01 00:00:00| Houston|    Texas|United States|Northwind 

In [65]:
df_our = spark.sql("SELECT * FROM `curated`.`sales`")

spark.conf.set("fs.azure.account.auth.type", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type", "com.microsoft.azure.synapse.tokenlibrary.ConfBasedSASProvider")
spark.conf.set("spark.storage.synapse.sas", "SAS Token")

path = "abfss://synfs@dataopsstg01.dfs.core.windows.net/c_ss.csv"

df_our.write.option("header",True).csv(path)

StatementMeta(sparkpool, 9, 64, Finished, Available)

In [63]:
%%sql
drop table curated.sales

StatementMeta(sparkpool, 9, 62, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>